In [1]:
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Found existing installation: tb-nightly 2.7.0a20211013
Uninstalling tb-nightly-2.7.0a20211013:
  Successfully uninstalled tb-nightly-2.7.0a20211013


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\nest_asyncio.py'
Consider using the `--user` option or check the permissions.

ERROR: Invalid requirement: '#'


In [2]:
import collections
import pickle
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import random
np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [3]:
def shuffling_client_taken(client_taken,client_percent, data_percent):
    client_data={}
    initial='clients'
    random_client = random.sample(range(0,client_taken), round(client_taken*client_percent))
    random_client = np.sort(random_client)
    print(random_client)
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    x=0
    y=0
    for i in range(client_taken):
        client_dataset = emnist_train.create_tf_dataset_for_client(
          emnist_train.client_ids[i])
        client_name=['{}_{}'.format(initial, i+1)]
        bla=np.zeros((1,784))
        sla= np.zeros((1,1))
        if random_client[y]==x:
            print("engaged")
            data_lenght=len(client_dataset)
            random_data= random.sample(range(0,data_lenght), round(data_lenght*data_percent))
            random_data= np.sort(random_data)
#             print(random_data)
            m=0
            n=0

            for example in client_dataset:
            # Append counts individually per label to make plots
            # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
#                 print(i,label,j)
                if random_data[n]==m:
                    label=[[random.randrange(0,9,1)]]
#                     label= label.reshape((-1,1))
                    label= np.array(label)
#                     print(label)
                    sla= np.concatenate((sla,label))
                    if n<len(random_data)-1:
                        n= n+1
#                     print(j)
                else:
                    sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                bla= np.concatenate((bla,data))
                m+=1
            if y<len(random_client)-1:
                y= y+1
        else:
            print('non_engaed')
            for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
                label = example['label'].numpy()
                label= label.reshape(-1,1)
                label= np.array(label)
                sla= np.concatenate((sla,label))
                data = example['pixels'].numpy()
                data = data.reshape(1,784)
                bla= np.concatenate((bla,data))
        x= x+1
        print(x,y)

                

        bla= bla[1:len(bla),:]
        sla= sla[1:len(sla),:]
#         sla= tf.keras.utils.to_categorical(sla, 10)
        dataset= list(zip(bla,sla))
        present_data= {client_name[0]:dataset}
        client_data.update(present_data)
    return client_data, random_client

In [4]:
def test_client_taken(client_taken):
    bla=np.zeros((1,784))
    sla= np.zeros((1,1))
    emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
    for i in range(client_taken):
        client_dataset = emnist_test.create_tf_dataset_for_client(
          emnist_test.client_ids[i])
        print('client taken {i}')
        
        for example in client_dataset:
        # Append counts individually per label to make plots
        # more colorful instead of one color per plot.
            label = example['label'].numpy()
            label= label.reshape(-1,1)
            label= np.array(label)
            sla= np.concatenate((sla,label))
            data = example['pixels'].numpy()
            data = data.reshape(1,784)
            bla= np.concatenate((bla,data))

    bla= bla[1:len(bla),:]
    sla= sla[1:len(sla),:]
#     sla= tf.keras.utils.to_categorical(sla, 10)
    return bla, sla

In [5]:
def save_file(file_name,data):
    open_file = open(file_name, "wb")
    pickle.dump(data, open_file)
    open_file.close()


In [15]:
#train data creation
num_client=3000
test_client= 3000
client_percent= .1
data_percent= 1
client_data, bad_client= shuffling_client_taken(num_client, client_percent,data_percent)


[  44   52   60   91   97  100  105  108  136  148  151  152  156  158
  167  168  173  178  189  204  213  214  227  237  270  275  286  299
  304  328  352  371  373  382  390  412  427  428  449  455  466  475
  477  495  503  529  537  548  559  585  592  599  606  614  620  626
  633  654  656  665  671  676  685  692  713  720  731  732  738  741
  757  797  821  824  831  834  844  848  853  856  864  886  887  888
  895  900  904  909  917  943  946  976  995 1004 1008 1035 1043 1061
 1070 1092 1094 1126 1129 1136 1140 1145 1179 1192 1211 1216 1228 1229
 1233 1244 1247 1253 1261 1262 1275 1279 1308 1328 1345 1357 1394 1412
 1415 1426 1428 1437 1448 1451 1472 1485 1500 1504 1521 1523 1550 1556
 1561 1562 1567 1570 1577 1582 1603 1608 1614 1616 1656 1659 1660 1669
 1671 1678 1695 1696 1703 1711 1719 1723 1733 1742 1748 1752 1761 1763
 1766 1770 1781 1787 1793 1796 1800 1803 1843 1867 1882 1888 1895 1917
 1925 1931 1940 1950 1956 1975 1979 1989 1998 2009 2029 2041 2046 2047
 2056 

393 35
non_engaed
394 35
non_engaed
395 35
non_engaed
396 35
non_engaed
397 35
non_engaed
398 35
non_engaed
399 35
non_engaed
400 35
non_engaed
401 35
non_engaed
402 35
non_engaed
403 35
non_engaed
404 35
non_engaed
405 35
non_engaed
406 35
non_engaed
407 35
non_engaed
408 35
non_engaed
409 35
non_engaed
410 35
non_engaed
411 35
non_engaed
412 35
engaged
413 36
non_engaed
414 36
non_engaed
415 36
non_engaed
416 36
non_engaed
417 36
non_engaed
418 36
non_engaed
419 36
non_engaed
420 36
non_engaed
421 36
non_engaed
422 36
non_engaed
423 36
non_engaed
424 36
non_engaed
425 36
non_engaed
426 36
non_engaed
427 36
engaged
428 37
engaged
429 38
non_engaed
430 38
non_engaed
431 38
non_engaed
432 38
non_engaed
433 38
non_engaed
434 38
non_engaed
435 38
non_engaed
436 38
non_engaed
437 38
non_engaed
438 38
non_engaed
439 38
non_engaed
440 38
non_engaed
441 38
non_engaed
442 38
non_engaed
443 38
non_engaed
444 38
non_engaed
445 38
non_engaed
446 38
non_engaed
447 38
non_engaed
448 38
non_engaed
4

non_engaed
859 80
non_engaed
860 80
non_engaed
861 80
non_engaed
862 80
non_engaed
863 80
non_engaed
864 80
engaged
865 81
non_engaed
866 81
non_engaed
867 81
non_engaed
868 81
non_engaed
869 81
non_engaed
870 81
non_engaed
871 81
non_engaed
872 81
non_engaed
873 81
non_engaed
874 81
non_engaed
875 81
non_engaed
876 81
non_engaed
877 81
non_engaed
878 81
non_engaed
879 81
non_engaed
880 81
non_engaed
881 81
non_engaed
882 81
non_engaed
883 81
non_engaed
884 81
non_engaed
885 81
non_engaed
886 81
engaged
887 82
engaged
888 83
engaged
889 84
non_engaed
890 84
non_engaed
891 84
non_engaed
892 84
non_engaed
893 84
non_engaed
894 84
non_engaed
895 84
engaged
896 85
non_engaed
897 85
non_engaed
898 85
non_engaed
899 85
non_engaed
900 85
engaged
901 86
non_engaed
902 86
non_engaed
903 86
non_engaed
904 86
engaged
905 87
non_engaed
906 87
non_engaed
907 87
non_engaed
908 87
non_engaed
909 87
engaged
910 88
non_engaed
911 88
non_engaed
912 88
non_engaed
913 88
non_engaed
914 88
non_engaed
915 8

1298 120
non_engaed
1299 120
non_engaed
1300 120
non_engaed
1301 120
non_engaed
1302 120
non_engaed
1303 120
non_engaed
1304 120
non_engaed
1305 120
non_engaed
1306 120
non_engaed
1307 120
non_engaed
1308 120
engaged
1309 121
non_engaed
1310 121
non_engaed
1311 121
non_engaed
1312 121
non_engaed
1313 121
non_engaed
1314 121
non_engaed
1315 121
non_engaed
1316 121
non_engaed
1317 121
non_engaed
1318 121
non_engaed
1319 121
non_engaed
1320 121
non_engaed
1321 121
non_engaed
1322 121
non_engaed
1323 121
non_engaed
1324 121
non_engaed
1325 121
non_engaed
1326 121
non_engaed
1327 121
non_engaed
1328 121
engaged
1329 122
non_engaed
1330 122
non_engaed
1331 122
non_engaed
1332 122
non_engaed
1333 122
non_engaed
1334 122
non_engaed
1335 122
non_engaed
1336 122
non_engaed
1337 122
non_engaed
1338 122
non_engaed
1339 122
non_engaed
1340 122
non_engaed
1341 122
non_engaed
1342 122
non_engaed
1343 122
non_engaed
1344 122
non_engaed
1345 122
engaged
1346 123
non_engaed
1347 123
non_engaed
1348 123


1720 161
non_engaed
1721 161
non_engaed
1722 161
non_engaed
1723 161
engaged
1724 162
non_engaed
1725 162
non_engaed
1726 162
non_engaed
1727 162
non_engaed
1728 162
non_engaed
1729 162
non_engaed
1730 162
non_engaed
1731 162
non_engaed
1732 162
non_engaed
1733 162
engaged
1734 163
non_engaed
1735 163
non_engaed
1736 163
non_engaed
1737 163
non_engaed
1738 163
non_engaed
1739 163
non_engaed
1740 163
non_engaed
1741 163
non_engaed
1742 163
engaged
1743 164
non_engaed
1744 164
non_engaed
1745 164
non_engaed
1746 164
non_engaed
1747 164
non_engaed
1748 164
engaged
1749 165
non_engaed
1750 165
non_engaed
1751 165
non_engaed
1752 165
engaged
1753 166
non_engaed
1754 166
non_engaed
1755 166
non_engaed
1756 166
non_engaed
1757 166
non_engaed
1758 166
non_engaed
1759 166
non_engaed
1760 166
non_engaed
1761 166
engaged
1762 167
non_engaed
1763 167
engaged
1764 168
non_engaed
1765 168
non_engaed
1766 168
engaged
1767 169
non_engaed
1768 169
non_engaed
1769 169
non_engaed
1770 169
engaged
1771 17

2139 206
non_engaed
2140 206
non_engaed
2141 206
non_engaed
2142 206
engaged
2143 207
non_engaed
2144 207
non_engaed
2145 207
non_engaed
2146 207
non_engaed
2147 207
non_engaed
2148 207
non_engaed
2149 207
non_engaed
2150 207
non_engaed
2151 207
non_engaed
2152 207
non_engaed
2153 207
engaged
2154 208
non_engaed
2155 208
engaged
2156 209
non_engaed
2157 209
non_engaed
2158 209
non_engaed
2159 209
non_engaed
2160 209
non_engaed
2161 209
engaged
2162 210
non_engaed
2163 210
non_engaed
2164 210
non_engaed
2165 210
non_engaed
2166 210
non_engaed
2167 210
non_engaed
2168 210
non_engaed
2169 210
non_engaed
2170 210
engaged
2171 211
non_engaed
2172 211
non_engaed
2173 211
non_engaed
2174 211
non_engaed
2175 211
non_engaed
2176 211
non_engaed
2177 211
non_engaed
2178 211
non_engaed
2179 211
non_engaed
2180 211
non_engaed
2181 211
non_engaed
2182 211
non_engaed
2183 211
non_engaed
2184 211
non_engaed
2185 211
non_engaed
2186 211
non_engaed
2187 211
non_engaed
2188 211
non_engaed
2189 211
non_en

2558 257
non_engaed
2559 257
non_engaed
2560 257
non_engaed
2561 257
non_engaed
2562 257
non_engaed
2563 257
engaged
2564 258
non_engaed
2565 258
non_engaed
2566 258
non_engaed
2567 258
non_engaed
2568 258
non_engaed
2569 258
non_engaed
2570 258
engaged
2571 259
non_engaed
2572 259
non_engaed
2573 259
non_engaed
2574 259
non_engaed
2575 259
non_engaed
2576 259
non_engaed
2577 259
non_engaed
2578 259
engaged
2579 260
non_engaed
2580 260
non_engaed
2581 260
non_engaed
2582 260
non_engaed
2583 260
non_engaed
2584 260
non_engaed
2585 260
non_engaed
2586 260
non_engaed
2587 260
non_engaed
2588 260
non_engaed
2589 260
non_engaed
2590 260
non_engaed
2591 260
non_engaed
2592 260
non_engaed
2593 260
non_engaed
2594 260
non_engaed
2595 260
non_engaed
2596 260
non_engaed
2597 260
non_engaed
2598 260
non_engaed
2599 260
non_engaed
2600 260
non_engaed
2601 260
non_engaed
2602 260
non_engaed
2603 260
engaged
2604 261
engaged
2605 262
non_engaed
2606 262
non_engaed
2607 262
non_engaed
2608 262
non_en

2979 298
non_engaed
2980 298
engaged
2981 299
non_engaed
2982 299
non_engaed
2983 299
non_engaed
2984 299
non_engaed
2985 299
non_engaed
2986 299
non_engaed
2987 299
non_engaed
2988 299
non_engaed
2989 299
engaged
2990 299
non_engaed
2991 299
non_engaed
2992 299
non_engaed
2993 299
non_engaed
2994 299
non_engaed
2995 299
non_engaed
2996 299
non_engaed
2997 299
non_engaed
2998 299
non_engaed
2999 299
non_engaed
3000 299


In [19]:
# test data creation
test_data, test_label= test_client_taken(num_client)

KeyError: 'Unable to open object (bad object header version number)'

In [16]:
file_name = f"Dataset_shuffle_{num_client}_{client_percent}_{data_percent}.pkl"

In [17]:
data=[client_data,bad_client]

In [18]:
save_file(file_name,data)